In [415]:
import pandas as pd
import json
from sqlalchemy import create_engine
import zipfile
import tarfile
import bz2
import sys
import os
import numpy as np


### As test data:
#### load only two files from every hour folder from the first day of the month

In [3]:
tweet_data = []

for folder in sorted(os.listdir("./data/archiveteam-twitter-stream-2020-06/06/01/")):
    if not folder.startswith('.'):
        print('load folder', folder)
        
        t = 0
        for file in sorted(os.listdir("./data/archiveteam-twitter-stream-2020-06/06/01/" + folder)):
            
            t += 1
            if file.endswith('.bz2'):
                
                print(t)
                print('load file', file)

                with bz2.open("./data/archiveteam-twitter-stream-2020-06/06/01/" + folder + "/" + file, "rt") as bzinput:


                    for i, line in enumerate(bzinput):
                        tweet = json.loads(line)
                        tweet_data.append(tweet)

            #include break statement to test
            # load only 2 files from each hour folder 
                if t == 2: break


load folder 00
1
load file 29.json.bz2
2
load file 30.json.bz2
load folder 01
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 02
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 03
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 04
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 05
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 06
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 07
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 08
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 09
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 10
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 11
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 12
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 13
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 14
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 15
1
load file 00.json.bz2
2
load file 01.j

In [3]:
len(tweet_data)

250819

### Lookup tables

In [4]:
# lookup table country names in different languages - country codes
df_c_multilingu = pd.read_csv('./data/country_names_multilingual/IP2LOCATION-COUNTRY-MULTILINGUAL.CSV')
df_c_multilingu.head()

,LANG,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_ALPHA3_CODE,COUNTRY_NUMERIC_CODE,COUNTRY_NAME
0,AF,AFRIKAANS,AD,AND,20,Andorra
1,AF,AFRIKAANS,AE,ARE,784,Verenigde Arabiese Emirate
2,AF,AFRIKAANS,AF,AFG,4,Afganistan
3,AF,AFRIKAANS,AG,ATG,28,Antigua en Barbuda
4,AF,AFRIKAANS,AI,AIA,660,Anguilla


,city,city_ascii,lat_3857,long_3857,COUNTRY_ALPHA2_CODE
0,Tokyo,Tokyo,35.6850,139.7514,JP
1,New York,New York,40.6943,-73.9249,US
2,Mexico City,Mexico City,19.4424,-99.1310,MX
3,Mumbai,Mumbai,19.0170,72.8570,IN
4,São Paulo,Sao Paulo,-23.5587,-46.6250,BR
5,Delhi,Delhi,28.6700,77.2300,IN
6,Shanghai,Shanghai,31.2165,121.4365,CN
7,Kolkata,Kolkata,22.4950,88.3247,IN
8,Los Angeles,Los Angeles,34.1139,-118.4068,US
9,Dhaka,Dhaka,23.7231,90.4086,BD


#### will i need to put in fuzzy matching or sth (umlauts etcs ASCII) transfer!!?!m

In [543]:
# lookup table country codes - coordinates
# file is from googlemaps

df_co_from_cc = pd.read_csv('./data/location_ref/countrycodes_coord.csv')

# drop nan rows
df_co_from_cc = df_co_from_cc.dropna(axis='rows')

# remove the second decimal places after the dot for valid latitude and logitude notation
df_co_from_cc['latitude'] = df_co_from_cc['latitude'].str.split('.').str[:-1].str.join('.')
df_co_from_cc['longitude'] = df_co_from_cc['longitude'].str.split('.').str[:-1].str.join('.')

# reame columns
df_co_from_cc = df_co_from_cc.rename({'country': 'COUNTRY_ALPHA2_CODE', 'latitude': 'lat_3857', 'longitude': 'long_3857'}, axis='columns')
df_co_from_cc['lat_long_3857'] = df_co_from_cc['lat_3857'] + ', ' + df_co_from_cc['long_3857']
df_co_from_cc.tail(20)

,COUNTRY_ALPHA2_CODE,lat_3857,long_3857,lat_long_3857
224,UA,48.379,3.116,"48.379, 3.116"
225,UG,1.373,32.290,"1.373, 32.290"
227,US,3.709,-95.712,"3.709, -95.712"
228,UY,-32.522,-55.765,"-32.522, -55.765"
229,UZ,41.377,64.585,"41.377, 64.585"
230,VA,41.902,12.453,"41.902, 12.453"
231,VC,12.984,-61.287,"12.984, -61.287"
232,VE,642,-6.658,"642, -6.658"
233,VG,18.420,-64.639,"18.420, -64.639"
234,VI,18.335,-64.896,"18.335, -64.896"


#### Google Maps uses a Mercator projection based on the World Geodetic System (WGS) 1984 geographic coordinate system (datum) == 3857

WGS 84 -- WGS84 - World Geodetic System 1984, used in GPS  == EPSG 4326m

In [544]:
# test the Transformer
from pyproj import Transformer
transformer = Transformer.from_crs('epsg:3857','epsg:4326')
transformer.transform(47.516, 14.550)

(0.000130704873839277, 0.0004268434904022318)

In [545]:
from pyproj import Transformer

def trans_func(lat_long_string) -> str:
    """transform coordinate system
    
    Parameters:
    ---------
    lat_long_string: e.g. '-38.416, -63.616'
    
    """    
    # instantiate transformer
    transformer = Transformer.from_crs('epsg:3857','epsg:4326')
    
    lat_str = lat_long_string.split(',')[0]
    long_str = lat_long_string.split(',')[1]
    transformed = transformer.transform(lat_str, long_str) # returns a tuple
    
    coords_str = (str(transformed[0])) + ', ' + (str(transformed[1]))
    return coords_str # in decimal degrees ! 
    
    

In [546]:
# apply function to df:
df_co_from_cc['coords_4326_dd'] = df_co_from_cc['lat_long_3857'].apply(trans_func)

In [547]:
df_co_from_cc.head(5)

,COUNTRY_ALPHA2_CODE,lat_3857,long_3857,lat_long_3857,coords_4326_dd
0,AD,42.546,1.601,"42.546, 1.601","1.4382027698753387e-05, 0.00038219722078149153"
1,AE,23.424,53.847,"23.424, 53.847","0.0004837158310340926, 0.0002104213721521567"
2,AF,3.393,67.709,"3.393, 67.709","0.0006082402957130625, 3.0479837590175363e-05"
3,AG,17.060,-61.796,"17.060, -61.796","-0.0005551229129658144, 0.00015325258747079032"
4,AI,18.220,-63.068,"18.220, -63.068","-0.0005665494833792671, 0.0001636730447665768"


### Get list of country names multilingu:

In [138]:
# filter for languages 
#https://www.loc.gov/standards/iso639-2/php/code_list.php
lang_4 = ['DE', 'FR', 'ES', 'SV']   

# EU + norway 
lang_european_un = ['DE', 'FR', 'HR', 'CS', 'DA', 'ET', 'FI', 
                    'EL',  'HU', 'GA', 'GL', 'IT', 'LT', 'LV', 'LB',
                    'SK', 'SL', 'NL', 'PL', 'PT', 'RO', 'SV', 'NN','ES', 'SV']

# assign current used languages list to "lang"
lang = lang_european_un

sel = df_c_multilingu[df_c_multilingu['LANG'].isin(lang)]
# # test
sel[sel['COUNTRY_NAME']=='Man']

c_nam_list = sel['COUNTRY_NAME'].to_list()

# remove problem causing country names 
c_nam_list.remove('Man')

# get unique country names
c_nam_list_unique = list(set(c_nam_list))
len(c_nam_list), len(c_nam_list_unique)

(5726, 3364)

### Functions

In [139]:
def get_from_country_coords(country_code) -> str:
    
    """Function to retrieve country coordinates from ISO ALPHA2 country code
    within a pandas dataframe"""
    
    country = df_co_from_cc[df_co_from_cc['COUNTRY_ALPHA2_CODE'] == country_code]
    # (lat, long) = (country['latitude'].to_list()[0], country['longitude'].to_list()[0])
    coords = country['coords_4326_dd'].to_list()[0]
    # Get country coords
    #return f'{lat}, {long}'
    return coords

In [140]:
# test
get_from_country_coords('IN')

'7.093097483405929e-05, 0.00018499006645873305'

In [141]:
def get_to_country_coords(c_name_multilingu) -> str:
    
    """Function to retrieve to-coordinates from country names that appear in the tweet text"""
    # assign detected country name to countrycode:
    
    code = (df_c_multilingu.loc[df_c_multilingu['COUNTRY_NAME'] == c_name_multilingu]['COUNTRY_ALPHA2_CODE'].unique())[0]
    # get coordinates for countrycode from lookup table via function:
    coords = get_from_country_coords(code)
    
    return coords # both coordinates as string

In [142]:
# test
#get_to_country_coords('Chine')

In [152]:
def get_user_loc_country_code():
    pass

In [143]:


def get_coordinates(bounding_coords, city_name, user_location) -> str:
    if bounding_coords:
        return bounding_coords
    elif city_name:
        # Lookup city name coordinates
        return '79.125472, 28.982011'
    elif user_location:
        # Lookup user location
        return '79.125472, 28.982011'
    elif country_code:
        # Lookup country code
        return get_from_country_coords(country_code)



### Write to postgres

In [144]:
# Declare postgres config
HOST = 'localhost'
USERNAME = 'postgres'
PORT = '5432'
DB = 'tweets_multilingu'
PASSWORD = 'postgres'

# Create a postgres connection and assign it to 'engine' variable
engine = create_engine(f'postgres://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}')

In [145]:
def extract_url(urls) -> str:
    
    """ Function to extract the url from the entities dict of the tweet json"""
    
    found_url = ''
    url_given = urls
    
    if url_given != None:
        for url in url_given:
            found_url = url.get('url')
            
    return found_url

In [166]:
user_locs = []
def load(data):
    """ Loads transformed tweeets into the Postgres database
    Parameters:
    ------------
    tweets: List of tweet dictionaries that were retrieved from the Twitter API """

    insert_query = """INSERT INTO tweets_test (
        tweet_id, 
        timestamp_ms,
        tweet, 
        url_in_tweet, 
        user_url, 
        user_location, 
        user_loc_country_code,
        user_followers_count, 
        user_verified, 
        user_statuses_count,
        user_id,
        user_created_at,

        lang,  
        from_coords,
        from_country_coords,
        to_country_coords,
        tweet_country_code)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
    
    counter = 0 
    counter_loc = 0 
    counter_country_ment = 0 
    # loop through list of jsons 
    for j in data:
        # check for valid entries and if tweet json contains any location information 
        valid_entry = j.get('created_at')
        user_loc = j.get('user', {}).get('location')
        place_attr = j.get('place')
        user_url = j.get('user', {}).get('url')
        urls = j.get('entities', {}).get('urls')
        
        # conditions to still be included ! 
        follower_c = j.get('user', {}).get('followers_count')
        ver = j.get('user', {}).get('verified')
        
        counter += 1
        
        if (valid_entry and len(urls) != 0 and user_url) and (user_loc or place_attr) and (ver or follower_c > 10000):
            
            #print(ver, follower_c)

            counter_loc += 1 
            
            # loop through multilingual country list 
            for c_name_multi in c_nam_list_unique:
                
                
                # check if one of the country names exist in tweet text
                if c_name_multi in j['text']:
                    counter_country_ment += 1
                    print(c_name_multi)

                    # get user dict
                    user = j.get('user')
                    # ger user_location if exits
                    user_location = user.get('location')
                    print(user_location)
                    user_locs.append(user_location)
                            
                    # get place dict if exists and extract first coordinate of the bounding box
                    place = j.get('place')
                    country_code = ''
                    tweet_country_code = ''
                    bounding_coords = ''
                    city_name = ''
                    from_coords = ''
                    from_country_coords = ''

                    if place != None:
                        country_code = place.get('country_code')
                        tweet_country_code = place.get('country_code')
                        
                        bounding_coords = ','.join(list(map(lambda n: str(n), place.get('bounding_box').get('coordinates')[0][0])))
                        
                        city_name = place.get('full_name')
                        from_coords = get_coordinates(bounding_coords, city_name, user_location)
                        from_country_coords = get_from_country_coords(country_code)
                        
                    to_country_coords = get_to_country_coords(c_name_multi)
                    print(to_country_coords)
                    
                    user_loc_country_code = 'XY'
                    
                    # get url mentionend in tweet from entities dict
                    found_url = extract_url(j.get('entities').get('urls'))
                    

                    # insert values into postgres DB                             
                    engine.execute(insert_query, (
                        j['id'], #id
                        j['timestamp_ms'], #timestamp_ms
                        j['text'], #tweet
                        found_url, #url_in_tweet
                        user['url'], #user_url
                        user_location, #user_location
                        user_loc_country_code,
                        user['followers_count'], #user_followers_count
                        user['verified'], #user_verified
                        user['statuses_count'], #user_statuses_count
                        user['id'], #user_id
                        user['created_at'], #user_created_at
                        
                        j['lang'], #lang
                        from_coords, #from_coords
                        from_country_coords, #from_country_coords
                        to_country_coords, #to_country_coords
                        tweet_country_code #tweet_country_code
                        )
         
                    )
    
    
    print('scanned jsons :', counter, 
          '\n with some location information and user url given :', counter_loc,
          '\n with some location information, user url and country name mentioned :',counter_country_ment)
    
     

In [307]:
load(tweet_data)

Granada
España
-0.0005533981476203856, 0.00011015142013873571
Hong Kong
Hong Kong
0.001025058587501262, 0.00020118669103140802
Japan
芝浜高校準軍事組織研究同好会
0.0012419388465036677, 0.00032522606546263154
Australia
Illinois, USA
0.0012017212712427817, -0.00022704020490836786
China
Illinois, USA
0.0009359996102467031, 3.221358608852604e-05
Japan
Osaka→Nara→Chiba(Tokyo)
0.0012419388465036677, 0.00032522606546263154
Peru
Wherever threads are written..
-0.0006738712103667231, -8.254619145774283e-05
França
Paris (France)
1.9879717237564612e-05, 0.0004152642063899311
Indonesia
All over the world
0.0010233697547673871, -0.007087707591703024
India
Worldwide
7.093097483405929e-05, 0.00018499006645873305
Laos
Bastille, Paris
0.0009207282504186758, 1.7831558389772503e-05
USA
Türkiye
-0.0008597955247042071, 3.331851388799305e-05
Cuba
España
-0.0006987186111236866, 0.00019332643229536223
España
El globo terráqueo
-0.003359699160681682, 0.00036348531341328196
India
Bengaluru, India
7.093097483405929e-05, 0.000

In [356]:
# lookup table world cities - coordinates
df_co_from_cities = pd.read_csv('./data/location_ref/simplemaps_worldcities_basicv1.6/worldcities.csv')
# has also the googlemaps EPSG!

df_co_from_cities = df_co_from_cities.rename({'iso2': 'COUNTRY_ALPHA2_CODE', 'lat': 'lat_3857', 'lng': 'long_3857'}, axis='columns')

# drop columns
df_co_from_cities = df_co_from_cities.drop(columns=['id', 'iso3', 'admin_name', 'country'])
df_co_from_cities.loc[20:30]

,city,city_ascii,lat_3857,long_3857,COUNTRY_ALPHA2_CODE,capital,population
20,Seoul,Seoul,37.5663,126.9997,KR,primary,9796000.0
21,Lagos,Lagos,6.4433,3.3915,NG,minor,9466000.0
22,Jakarta,Jakarta,-6.1744,106.8294,ID,primary,9125000.0
23,Guangzhou,Guangzhou,23.1450,113.3250,CN,admin,8829000.0
24,Chicago,Chicago,41.8373,-87.6862,US,NaN,8675982.0
25,London,London,51.5000,-0.1167,GB,primary,8567000.0
26,Lima,Lima,-12.0480,-77.0501,PE,primary,8012000.0
27,Tehran,Tehran,35.6719,51.4243,IR,primary,7873000.0
28,Kinshasa,Kinshasa,-4.3297,15.3150,CD,primary,7843000.0
29,Bogotá,Bogota,4.5964,-74.0833,CO,primary,7772000.0


In [357]:
df_co_from_cities.shape

(15493, 7)

In [358]:
# look for duplicates
# df is ordered: the cityname with the highest population number is the first
df_co_from_cities[df_co_from_cities['city'] == 'Paris']

,city,city_ascii,lat_3857,long_3857,COUNTRY_ALPHA2_CODE,capital,population
19,Paris,Paris,48.8667,2.3333,FR,primary,9904000.0
9170,Paris,Paris,33.6689,-95.5462,US,NaN,25961.0
11195,Paris,Paris,38.2016,-84.2719,US,NaN,12662.0
11815,Paris,Paris,36.2933,-88.3065,US,NaN,10576.0
12546,Paris,Paris,39.6148,-87.6905,US,NaN,8431.0
14992,Paris,Paris,44.2445,-70.4883,US,NaN,5148.0


In [359]:
df_co_from_cities[df_co_from_cities['city'] == 'London']

,city,city_ascii,lat_3857,long_3857,COUNTRY_ALPHA2_CODE,capital,population
25,London,London,51.5000,-0.1167,GB,primary,8567000.0
1388,London,London,42.9700,-81.2500,CA,NaN,346765.0
8388,London,London,37.1209,-84.0804,US,NaN,37424.0
10741,London,London,39.8936,-83.4375,US,NaN,14597.0


In [360]:
# drop the non-major cities with the same name as other capital/major cities
df_co_from_cities = df_co_from_cities.drop_duplicates(subset=['city'], keep='first')
df_co_from_cities.shape

(13504, 7)

In [361]:
# check if duplicates still exist
duplicates = df_co_from_cities[df_co_from_cities.duplicated(subset=['city'])]
duplicates.shape # no duplicates left

(0, 7)

In [362]:
user_locs

['Shenzhen',
 'London',
 'España',
 'Hong Kong',
 '芝浜高校準軍事組織研究同好会',
 'Illinois, USA',
 'Illinois, USA',
 'Osaka→Nara→Chiba(Tokyo)',
 'Wherever threads are written..',
 'Paris (France)',
 'All over the world',
 'Worldwide',
 'Bastille, Paris',
 'Türkiye',
 'España',
 'El globo terráqueo',
 'Bengaluru, India',
 'Japan',
 'Nairobi, Kenya',
 'London',
 'Port de Jaffa, Israel',
 'kursusutarih@gmail.com ',
 'España',
 'España',
 'New Delhi, India',
 'Rome',
 'Buenos Aires, Argentina',
 'Qatar',
 'Porto Alegre',
 'Worldwide',
 'Worldwide',
 'Worldwide',
 'New England',
 'México, D.F.',
 'México',
 'Indonesia, Jakarta',
 'Outside the Bubble...',
 'San Salvador, El Salvador',
 'San Salvador, El Salvador',
 'London, UK',
 'Base del Comando Gral. Cocolí.',
 'Nashville, TN  USA',
 'Sede Centrale - ROMA',
 'Córdoba, Argentina',
 'Mendoza - Argentina',
 'Distrito Federal, México',
 'Beirut, Líbano',
 'Beirut, Líbano',
 'New York, NY',
 'Panama',
 'Colombia',
 'Africa',
 'Africa',
 'Ciudad Universita

In [419]:
# seriesP
for user_loc in user_locs:
    #print(user_loc)
    f = (s[s.isin([user_loc])]).values
    if f.size:   # numpy.ndarray.size :Number of elements in the array.
        print(f)


['Shenzhen']
['London']
['Hong Kong']
['London']
['Rome']
['Porto Alegre']
['Chicago']
['Maracaibo']


### some user locations have the country name instead of the city! can also be checked and assigned coordinates with the same table ref! 

### account for city names in different languages --> need another lookup table 

In [439]:
for cityname in s:
    #print(cityname)
    match = user_locs_s[user_locs_s.str.contains(cityname)]
    if match.size:   # numpy.ndarray.size :Number of elements in the array.
        print(match.values)
    valid_cityname = user_locs_s.at[user_locs_s.str.contains(cityname)]
    print(valid_cityname)

['Osaka→Nara→Chiba(Tokyo)']


ValueError: Invalid call for scalar access (getting)!

In [479]:
city_ind = s[s.str.contains('Berlin')].index.values[0].item()


city_ind

96

In [480]:
s.loc[city_ind]

'Berlin'

In [490]:
user_locs_s = pd.Series(user_locs) 
#city_ind = user_locs_s[user_locs_s.str.contains('Tokyo')].index.values[0]
#city_ind

7

In [542]:
for cityname in s:
    found_city = user_locs_s[user_locs_s.str.contains(cityname)].values
    if found_city.size > 0:
        print(s[s==cityname].index.values[0])
#         print(cityname.index)

0
1
5
10
19
22
24
25
26
30
32
40
50
73
75
93
98
116
119
195
204
231
257
325
333
360
455
541
673
821
885
900
1161
1885
2301
3345
3507
4296
5090
5498
5718
6985
8228
8822
8899
10421
13060
13584
13839


In [ ]:
s